In [1]:
import geopandas
import pandas
import pathlib
import shapely
import numpy
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
base_path = pathlib.Path(r"C:\Local\data\river_bathemetry\sections\Westport")

# Split out different sections depending on the test dataset
### Small
Includes sections Section7, Section8, Section9, Section10

In [3]:
sections = geopandas.read_file(base_path / "river_outlines" / "small" / "measured_sections.geojson")
riverbanks = geopandas.read_file(base_path / "river_outlines" / "small" / "river_banks.geojson")

In [4]:
riverbanks

,Name,geometry
0,Right,"LINESTRING (1483413.197 5375259.983, 1483663.4..."
1,Left,"LINESTRING (1483187.524 5375194.332, 1483229.3..."


# Explore Interpolation between sections
1. Create a centreline from the polygon
2. Create cross sections along the centreline
3. Estimate the Thalweg from the measued sections
4. Interpolate along points sampled along the lines

### 1. Create Centreline from briverbanks file
* Resample to the same number of nodes
* Explode into a row for each node
* Create a centreline (again same number of points)

In [5]:
# resample to the same number of points
n = 100
for row in range(len(riverbanks)):
    riverbanks.loc[row, 'geometry'] = shapely.geometry.LineString([riverbanks.iloc[row].geometry.interpolate(float(i)/n, normalized=True) for i in range(n + 1)])

In [6]:
#riverbanks['Points'] = riverbanks.apply(lambda row: row['geometry'].coords, axis=1)
riverbanks['points'] = riverbanks.apply(lambda row: shapely.geometry.MultiPoint([pt for pt in row['geometry'].coords]), axis=1)
riverbanks = riverbanks.set_geometry('points', drop=True).explode(index_parts=True)
riverbanks = pandas.concat([riverbanks[riverbanks['Name']=='Right'].droplevel(0).geometry.rename('Right'), 
                            riverbanks[riverbanks['Name']=='Left'].droplevel(0).geometry.rename('Left')], axis = 1)
riverbanks['centre'] = riverbanks.apply(lambda row: shapely.geometry.MultiPoint([row['Right'], row['Left']]).centroid , axis=1)

In [7]:
riverbanks

,Right,Left,centre
0,POINT (1483413.197 5375259.983),POINT (1483187.524 5375194.332),POINT (1483300.361 5375227.157)
1,POINT (1483420.667 5375241.345),POINT (1483194.241 5375175.629),POINT (1483307.454 5375208.487)
2,POINT (1483428.137 5375222.708),POINT (1483200.958 5375156.926),POINT (1483314.547 5375189.817)
3,POINT (1483435.606 5375204.070),POINT (1483207.676 5375138.223),POINT (1483321.641 5375171.147)
4,POINT (1483443.076 5375185.433),POINT (1483214.393 5375119.520),POINT (1483328.734 5375152.477)
...,...,...,...
96,POINT (1483987.870 5373443.502),POINT (1483792.970 5373413.434),POINT (1483890.420 5373428.468)
97,POINT (1483989.658 5373423.503),POINT (1483793.425 5373393.566),POINT (1483891.542 5373408.535)
98,POINT (1483991.447 5373403.504),POINT (1483793.880 5373373.699),POINT (1483892.663 5373388.602)
99,POINT (1483993.235 5373383.506),POINT (1483794.335 5373353.832),POINT (1483893.785 5373368.669)


### 2. Create cross sections along the centreline
* Calculate width
* Calculate the normal at each centreline point
* Create cross sections to width

In [17]:
riverbanks['Right'].iloc[0].x

1483413.1971816276

In [23]:
# Distance by numpy
import time
print("Compare shapely and numpy time")
start = time.time()
print(f"numpy max distance {numpy.max([numpy.sqrt((pt1.x - pt2.x)**2 + (pt1.y - pt2.y)**2) for (pt1, pt2) in zip(riverbanks['Right'], riverbanks['Left'])])}")
end = time.time()
print(f"time is {end - start}")
# Distance by shapely
start = time.time()
print(f"shapely time {numpy.max([pt1.distance(pt2) for (pt1, pt2) in zip(riverbanks['Right'], riverbanks['Left'])])}")
end = time.time()
print(end - start)
print("numpy is slower than shapely")

Compare shapely and numpy time
numpy max distance 308.73408146871725
time is 0.007758378982543945
shapely time 308.73408146871725
0.0025522708892822266
numpy is slower than shapely


In [25]:
riverbanks["width"] = riverbanks.apply(lambda row: row['Left'].distance(row['Right']), axis=1)

In [ ]:
def generate_cross_section(width, slope, centre):
    """ Create a line segment out from the centre point that defines the cross section """
    return
def calculate_slope():
    """ Calculate the slope along the centre points. try apply and smoothing for the 
    forward and backward segment and then combine as an average ignoring NaN"""
    return

In [ ]:
def _segment_slope(x_array, y_array, index):
    """Return the slope and length characteristics of a line segment.

    Parameters
    ----------

    x_array
        The x values of all polyline nodes.
    y_array
        The y values of all polyline nodes.
    index
        The segment index (the index of the starting node in the segment)
    """
    length = numpy.sqrt(
        (x_array[index + 1] - x_array[index]) ** 2
        + (y_array[index + 1] - y_array[index]) ** 2
    )
    dx = (x_array[index + 1] - x_array[index]) / length
    dy = (y_array[index + 1] - y_array[index]) / length
    return dx, dy, length

In [ ]:
# Set width of each 
centrelines = {"widths": [], "cross sections": []}
for (pt1, pt2) in zip(riverbanks.iloc[0].geometry.coords, riverbanks.iloc[1].geometry.coords):
    centrelines["widths"].append(numpy.max([numpy.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)]))
    # Calculate perpindiculars ato 
    centrelines["cross sections"].append(shapely.geometry.LineString([[] for i in range(n+1)]))
    

In [ ]:
# Set width of each 
centrelines = {"widths": [], "cross sections": []}
(x_array, y_array) = riverbanks.iloc[2].geometry.xy
for index, (pt1, pt2) in enumerate(zip(riverbanks.iloc[0].geometry.coords, riverbanks.iloc[1].geometry.coords)):
    centrelines["widths"].append(numpy.max([numpy.sqrt((pt1[0] - pt2[0])**2 + (pt1[1] - pt2[1])**2)]))
    # Calculate perpindiculars ato 
    #centrelines["cross sections"].append(shapely.geometry.LineString([[] for i in range(n+1)]))
    if index == 0:
        # first segment - slope of next segment
        dx, dy, length = _segment_slope(x_array, y_array, index)
    elif index == len(x_array) - 1:
        # last segment - slope of previous segment
        dx, dy, length = _segment_slope(x_array, y_array, index - 1)
    else:
        # slope of the length weighted mean of both segments
        dx_prev, dy_prev, l_prev = _segment_slope(x_array, y_array, index)
        dx_next, dy_next, l_next = _segment_slope(x_array, y_array, index - 1)
        dx = (dx_prev * l_prev + dx_next * l_next) / (l_prev + l_next)
        dy = (dy_prev * l_prev + dy_next * l_next) / (l_prev + l_next)
        length = (l_prev + l_next) / 2
    normal_x = -dy
    normal_y = dx
    cross_sections_dict["geometry"].append(
                shapely.geometry.LineString(
                    [
                        [
                            x_array[i] - self.transect_radius * normal_x,
                            y_array[i] - self.transect_radius * normal_y,
                        ],
                        [x_array[i], y_array[i]],
                        [
                            x_array[i] + self.transect_radius * normal_x,
                            y_array[i] + self.transect_radius * normal_y,
                        ],
                    ]
                )
            )
    centrelines["cross sections"].append(shapely.geometry.LineString([[] for i in range(n+1)]))

In [ ]:
if i == 0:
                # first segment - slope of next segment
                dx, dy, length = self._segment_slope(x_array, y_array, i)
            elif i == len(x_array) - 1:
                # last segment - slope of previous segment
                dx, dy, length = self._segment_slope(x_array, y_array, i - 1)
            else:
                # slope of the length weighted mean of both segments
                dx_prev, dy_prev, l_prev = self._segment_slope(x_array, y_array, i)
                dx_next, dy_next, l_next = self._segment_slope(x_array, y_array, i - 1)
                dx = (dx_prev * l_prev + dx_next * l_next) / (l_prev + l_next)
                dy = (dy_prev * l_prev + dy_next * l_next) / (l_prev + l_next)
                length = (l_prev + l_next) / 2
            normal_x = -dy
            normal_y = dx

In [ ]:
centrelines["widths"]

# debug geofabrics

In [ ]:
cache_path = pathlib.Path(r"C:\Local\data\catchments\Westport\caches\NZ20_Westport")
folder = "paper"
channel = geopandas.read_file(cache_path / folder / "wide_river" / "network_river_centreline_6308000000_smoothed.geojson")

In [ ]:
channel2 = geopandas.read_file(cache_path / folder / "wide_river" / "aligned_river_centreline_6308000000.geojson")

In [ ]:
geopandas.read_file(r"C:\Local\repos\GeoFabrics\tests\test_river_bathymetry_osm_wellington\data\

In [ ]:
n=geopandas.read_file(r"C:\Local\repos\GeoFabrics\tests\test_river_bathymetry_osm_wellington\data\rec1_flow_and_friction.geojson").to_crs(crs)

In [ ]:
crs

In [ ]:
n

In [ ]:
channel.rotate(45).plot()

In [ ]:
channel.rotate(45).boundary.plot()